In [3]:
import tensorflow as tf

# 定义计算滑动平均的变量[必须是实数型变量]，初始值为0
v1 = tf.Variable(0, dtype=tf.float32)
# 定义模拟神经网络迭代轮数，可用于动态控制衰减率，不被训练
step = tf.Variable(0, trainable=False)

# 定义滑动平均的类，初始衰减率0.99，控制衰减率的变量step
ema = tf.train.ExponentialMovingAverage(decay=0.99, num_updates=step)
# 定义更新变量滑动平均的操作
# 这里给定是一个列表，每次apply(update_var_list)执行是更新列表中变量
maintain_averages_op = ema.apply([v1])

with tf.Session() as sess:
    # 初始化所有变量
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    # average()函数获取滑动平均之后变量的值
    # 滑动平均不会改变变量本身的取值，而是维护一个shadow来记录滑动平均值
    # 当需要使用这个滑动平均值的时候，需要明确调用average()函数
    print(sess.run([v1, ema.average(v1)]))
    
    # 更新变量v1为5
    sess.run(tf.assign(v1, 5))
    # 更新v1的滑动平均值
    # 衰减率为min{0.99, (1+step)/(10+step)}
    sess.run(maintain_averages_op)
    print(sess.run([v1, ema.average(v1)]))
    
    # 继续更新
    sess.run(tf.assign(step, 10000))
    sess.run(tf.assign(v1, 10))
    sess.run(maintain_averages_op)
    print(sess.run([v1, ema.average(v1)]))
    
    # 继续更新
    sess.run(maintain_averages_op)
    print(sess.run([v1, ema.average(v1)]))

[0.0, 0.0]
[5.0, 4.5]
[10.0, 4.5549998]
[10.0, 4.6094499]
